In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
import matplotlib.pyplot as plt

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Set dataset root directory
dataset_root = './data'

# Create dataset objects
trainset = datasets.CIFAR100(dataset_root, train=True, transform=data_transforms, download=True)
testset = datasets.CIFAR100(dataset_root, train=False, transform=data_transforms, download=True)

# Create data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:03<00:00, 46.2MB/s]


In [4]:
# Set number of classes
num_cls = 100

# Load VGG16 with pretrained weights
model = models.vgg16(pretrained=True)

# Extract the number of input features for the last fully connected layer
num_in_ftrs = model.classifier[6].in_features

# Replace the last fully connected layer
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls)

# Freeze all layers except the last one
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier[6].parameters():
    param.requires_grad = True

# Move model to device
model = model.to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Create optimizer
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Number of epochs
num_epochs = 10

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 146MB/s]


In [5]:
# To track the best model
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = trainloader
        else:
            model.eval()   # Set model to evaluation mode
            dataloader = testloader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Deep copy the model if it's the best one so far
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    print()

print(f'Best val Acc: {best_acc:.4f}')



Epoch 1/10
----------
train Loss: 2.3243 Acc: 0.4281
val Loss: 1.6698 Acc: 0.5559

Epoch 2/10
----------
train Loss: 1.7072 Acc: 0.5320
val Loss: 1.5036 Acc: 0.5874

Epoch 3/10
----------
train Loss: 1.5961 Acc: 0.5522
val Loss: 1.4450 Acc: 0.5970

Epoch 4/10
----------
train Loss: 1.5323 Acc: 0.5694
val Loss: 1.4051 Acc: 0.6010

Epoch 5/10
----------
train Loss: 1.4866 Acc: 0.5783
val Loss: 1.3794 Acc: 0.6063

Epoch 6/10
----------
train Loss: 1.4643 Acc: 0.5844
val Loss: 1.3501 Acc: 0.6169

Epoch 7/10
----------
train Loss: 1.4350 Acc: 0.5889
val Loss: 1.3350 Acc: 0.6145

Epoch 8/10
----------
train Loss: 1.3757 Acc: 0.6056
val Loss: 1.3179 Acc: 0.6213

Epoch 9/10
----------
train Loss: 1.3584 Acc: 0.6132
val Loss: 1.3140 Acc: 0.6229

Epoch 10/10
----------
train Loss: 1.3558 Acc: 0.6116
val Loss: 1.3060 Acc: 0.6257

Best val Acc: 0.6257


In [6]:
# Load best model weights
model.load_state_dict(best_model_wts)

# Save the best model
torch.save(model.state_dict(), 'best_model_vgg16.pth')

# Test the model
model.eval()   # Set model to evaluation mode

# Tracking variables
correct = 0
total = 0

# No gradient needed for testing
with torch.no_grad():
    for inputs, labels in testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 62.39%
